# Review 정제, 직접 정의한 feature로 벡터화, cosine distance 계산
1. review를 okt(구 트위터 형태소 분석기)와 kkma(꼬꼬마 형태소 분석기)를 활용해서 전처리한다. morphy함수 사용.
2. feature를 list 형식으로 정의한다. 유의어 계속 추가해야함.
3. morphy한 형태소에서 feature를 찾아내서 정규화
4. 정규화 된 벡터끼리 코사인 거리 측정

2019_01_16

In [1]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from konlpy.tag import Kkma
import numpy as np
import os
from scipy import spatial

## feature와 stopwords

In [13]:
#feature lists : 각 feature의 값을 결정할 단어(유의어) 모음
sad = ['사고', '사망','고독','자살', '알코올', '신파', '상처', '치유', '슬프','초라','비극','통탄', '애석', '원통', '비참', '애통', '애처로', '우울', '아프', '서럽', '침통', '비통', '비탄', '애틋', '서글프', '원통', '섧', '눈물', '침울', '무력', '버림받', '그림자', '그늘', '걱정', '근심', '시한부', '이별', '감성적', '싹트']

horror = ['무섭', '목숨', '오싹', '긴장', '추격', '목숨','오싹', '시체', '잡아먹', '엽기', '정체불명', '묶이', '자살', '고문','살해', '괴담', '탈출', '살점', '끔찍하', '찌푸리', '질색', '죽이려','공포', '불안', '경악', '두려', '긴장', '숨통', '호러', '무서', '겁','주저', '오금', '가위', '악몽', '불안전', '위협', '살인', '살벌', '소름', '귀신', '괴물', '살인마', '실종', '흉가', '잔인','잔혹', '참혹', '참상','고어', '하드코어', '끔찍', '징그러', '흉하', '소름', '역겨', '벌레', '썩은', '흉측', '살인', '쏘우', '복수', '구역질', '토', '피', '살인','두려움']

action = ['악당', '미션', '스파이', '간첩', '007', '경찰','대역', '스턴트', '액션', '대규모', '세트장', '헐리우드', '훈련' ,'격투',]

family = [ '보호', '따뜻', '가정적','임신', '고양이', '키우던', '애정', '각별', '동거', '홀로', '고향', '자살', '친엄마', '힐링', '가족적', '우애', '형제', '자매', '남매', '교훈', '가문', '물려받', '노총각','고명딸','명절', '휴먼', '성탄절','크리스마스이브','남동생','성장','소년','소녀','수련','가족','일족','집','혈통','종족','가구','친척','혈연','관계','자손','가정','아들','딸', '아버지', '어머니','아빠','엄마','부부','남편','아내','결혼','러브','이혼','임신','불임', '할아버지', '할머니', '손자', '손녀', '손주', '이모', '삼촌', '조카', 'love', 'affair', '감동', '입양']

SF= ['세기', '태양계', '종족', '로봇', '재난', '우주인', '산소','지구', '우주','미래','과학','외계인','행성', '블랙홀','가상']

love = ['인연', '보호', '따뜻', '동거', '각별', '임신', '멜로', '운명적', '만남', '짝사랑', '감동','로맨스','사랑','연인','호감','소중','매력','로맨틱','러브','애정','첫눈','이끌리','감정','남녀','관계','남편','아내','친분','재회','첫만남','결혼','러브','동성','성적','이별','애원','섹스','마음', '첫사랑', '행복', '낭만적', '연애관']

hero = ['거대','화려','넘치','장엄','현란','압도적','아이맥스', '블록버스터', '특수', '스케일', '분장','효과', '활약', '파트너','정의', '종말', '영웅','용맹','용기','대담','불굴','씩씩','히어로','마블','디씨','캡틴','리더','지도자','인도자','구출','악당', 'DC', 'Marvel']

comedy = ['노총각', '유머러스하', '재미있', '코미디', '유머', '명절', '웃기', '재미있는', '웃으', '유쾌', '떠들썩', '즐거운','코미디언',]

queer = ['동성애', '게이', '레즈', '레즈비언', '트렌스젠더', '퀴어', '동성']

feminism = ['여권','여성','페미니즘', '양성', '페미']

time_travel = ['타임', '돌아가', '퓨처', '타임머신', '시간여행', '미래']

school = ['성적','청춘','성장', '명량', '고딩', '고등학생', '고등학교', '학교', '성장통', '성장기', '선생님', '학생', '교생', '교복', '사춘기', '짝사랑', '학원물', '청춘', '첫사랑','대학','청년']

kid = ['아동용', '공주','애니', '극장판', '애니메이션', '캐릭터', '성우', '더빙', '어린이', '초등학생', '만화', 'TV시리즈','디즈니']

docu = ['박근', '진지', '다큐', '다큐멘터리', '실화','현실','기록']

disaster = ['사고', '대책', '안보', '감염', '좀비', '바이러스', '사살', '시신', '생존자', '살아남', '목숨', '죽음', '희생', '지구', '진상', '재난', '재해', '구조', '재앙', '멸망', '폭풍', '침몰', '전쟁', '대피']

political = ['박근', '노무', '혁명', '정부', '시위', '투표', '정권', '퇴진', '정치', '노무현', '공화국', '민주주의', '독재', '대통령', '인권', '참정권', '운동', '노예']

war = ['국방장관', '군부', '안보', '공산당', '대책', '미사일', '기지', '소련', '미군', '공군', '해군', '전투', '전우애', '전선', '휴전', '대대장', '북진', '남하', '부상병', '탱크', '폭탄', '냉전', '아군', '적군', 'GP', '초소', '전쟁', '소대원', '이병', '일병', '상병', '병장', '북한', '총', '칼', '공습']

thriller = ['추격전', '쫓겨나', '어둠', '죽이', '서스펜스', '박진','무섭', '스릴러',]

crime = ['추격전', '사기', '죽이', '고스톱', '도박', '도박장','살인', '청부', '욕망', '파멸', '추락', '알코올', '중독', '심문', '강력계', '범죄', '마약', '도둑', '조폭', '형사', '단속','불법','격투','범죄','마약',] 

twist = ['반전', '반전있']

music= ['교향곡','악장','피아노','선율','재즈','연주','밴드','뮤지컬','음악','리드미컬하','리드미컬','멜로디','공연','무대','악기','조율','대중가요','노래','건반','기타','안무']

genius=['두뇌','게임','베팅','배팅','설계','경기','대학','거물','카드','도박','천재','아이비',]

fun=['흥미진진','재미있','재미']

fantasy=['반지', '요술', '지팡이', '마법', '빗자루', '엘프', '제왕', '오크', '주문', '마법사', '판타지', '드래곤', '롤링', '워프', '해리']


stops = ["★★★★★", "★★★★", "★★★", "★★", "★", '있', '하', '것', '들', '그', '되', '수', '이', '보', '않', '없', '나', '사람', '주', '아니', '등', '같', '우리', '때', '년', '가', '한', '지', '대하', '오', '말', '일', '그렇', '위하', '때문', '그것', '두', '말하', '알', '그러나', '받', '못하', '일', '그런', '또', '문제', '더', '사회', '많', '그리고', '좋', '크', '따르', '중', '나오', '가지', '씨', '시키', '만들', '지금', '생각하', '그러', '속', '하나', '집', '살', '모르', '적', '월', '데', '자신', '안', '어떤', '내', '내', '경우', '명', '생각', '시간', '그녀', '다시', '이런', '앞', '보이', '번', '나', '다른', '어떻', '개', '전', '들', '사실', '이렇', '점', '싶', '말', '정도', '좀', '원', '잘', '통하', '소리', '놓']





featureVector.append(featureDetector(sad, morphs))
featureVector.append(featureDetector(horror, morphs))
featureVector.append(featureDetector(action, morphs))
featureVector.append(featureDetector(family, morphs))
featureVector.append(featureDetector(SF, morphs))
featureVector.append(featureDetector(love, morphs))
featureVector.append(featureDetector(hero, morphs))
featureVector.append(featureDetector(comedy, morphs))
featureVector.append(featureDetector(queer, morphs))
featureVector.append(featureDetector(feminism, morphs))
featureVector.append(featureDetector(time_travel, morphs))
featureVector.append(featureDetector(school, morphs))
featureVector.append(featureDetector(kid, morphs))
featureVector.append(featureDetector(docu, morphs))
featureVector.append(featureDetector(disaster, morphs))
featureVector.append(featureDetector(political, morphs))
featureVector.append(featureDetector(war, morphs))
featureVector.append(featureDetector(thriller, morphs))
featureVector.append(featureDetector(crime, morphs))
featureVector.append(featureDetector(twist, morphs))
featureVector.append(featureDetector(music, morphs))
featureVector.append(featureDetector(genius, morphs))
featureVector.append(featureDetector(fun, morphs))
featureVector.append(featureDetector(fantasy, morphs))



NameError: name 'featureVector' is not defined

In [4]:
# feature 항목 형태소화
def featurizer(feat):
    featStr = ','.join(feat)
    feat = morphy(featStr)
    feat = set(feat)
    print(feat)
    return feat

In [5]:
#stopword: 2글자 이상 단어 중 필요 없는 단어 계속 추가해서 필터링!
stop = ["있다","다는","은데","특히","있었","동안","면서","을까","해하","어떤","한때","어야","듯이","ㄴ다",'Story',"cinepark","co","kr","Review"]

### morphy : 리뷰에서  형태소 분석

In [6]:
def morphy(review):
    kkma = Kkma()
    morphs = kkma.morphs(review)
    morphs = set(morphs)
    morphs = [w for w in morphs if ((w not in stop) and (len(w)>1))]#or w == '.' or w == '!' or w == '?'))]
    
    return morphs

'''    # 나뉜 형태소를 문장별로 list화 하기
    sentences = []
    
    idx = 0
    i = 0
    size = len(morphs)
    
    while(i < size ):
        sentence = []
        while(True):
            if(morphs[i] == "." or i == size-1 or morphs[i] == "!" or morphs[i] == "?"):
                break
            sentence.append(morphs[i])
            i += 1
        sentences.append(sentence)
        i += 1
        return  sentences'''

### feature detecter : 
주어진 feature내 유사어를 형태소에서 찾아서 수를 리턴

In [7]:
#feat은 sad, horror 같은 feature 유의어 사전
#mor는 사용할 morphs (리뷰)
def featureDetector(feat, mor):
    cnt = 0
    for w in mor:
        if w in feat:
            cnt+=1
    return cnt

### feature Normalizer:
Detect한 feature를 정규화

In [8]:
def featureNormalization(feature):
    total = 0
    for i in range(len(feature)):total += feature[i]
    for i in range(len(feature)):feature[i] /= total
    return feature

### feture vectorizer :
추출한 feature를 해당 영화의 feature vector화 한다.
추가로 정규화까지!

In [12]:
def featureVectorizer(morphs):
    
    featureVector = []
    
    featureVector.append(featureDetector(sad, morphs))
    featureVector.append(featureDetector(horror, morphs))
    featureVector.append(featureDetector(action, morphs))
    featureVector.append(featureDetector(family, morphs))
    featureVector.append(featureDetector(futuristic, morphs))
    featureVector.append(featureDetector(love, morphs))
    featureVector.append(featureDetector(hero, morphs))
    featureVector.append(featureDetector(comedy, morphs))
    
    #정규화
    featureNormalization(featureVector)
    
    return featureVector

# 예시

In [29]:
review1 = open('data\cine21\프리즈 프레임(2004).txt',encoding='UTF8').read()
review2 = open('data\cine21\플레이스 비욘드 더 파인즈(2012).txt',encoding='UTF8').read()

In [31]:
%time morphs1 = morphy(review1)
%time morphs2 = morphy(review2)

Wall time: 831 ms
Wall time: 570 ms


In [32]:
#전처리 결과
print(morphs1)
print(morphs2)

['몰살', '오로지', '일가족', '기록하', '바야흐로', '하여', '뒤집어씌우', '갑자기', '심리', '움직이', '시대', '부터', '21', '배경', '방송국', '무너지', '누구', '케이', '익숙', '기억', '그러', '측면', '고정', '모습', '단계', '법정', '범죄', '총동원', '음미', '시원', '만년필', '공산', 'ㄹ까', '빼어나', '궁금증', '만하', '등장', '험악', '세기', '제한', '단면', '자신', '시각적', '위치', '으나', '레이첼', '프레임', '온데간데없', '90', '15', '이야기', '모자라', '파라', '알리바이', '혹은', '불충분', '이미지', '기자', '들이닥치', '우의', '흔들리', '공이', '개연성', '마구', '재스퍼', '을러대', '조금', '다그치', '압축', '풀려나', '후반부', '그것', '오히려', '라고', '그날', '전제', '스토리', '다르', '언행', '수단', '악역', '유발', '훌쩍', '결백', '인물', '대비', '자동적', '로서', '스릴러', '들어오', '주체', '어느', '단점', '복제', '못하', '외출', '들어가', '편집증', '혐의', '문제', '에머릭', '나가', '등장인물', '핵심', '에게', '보이', '메리', '10', '갑작스럽', '안전', '저장', '카메라', '답습', '형사', '그리', '증거', '프리즈', '나오', '살인범', '몰리', '부조리', '시공간', '기계', '정말', '다가오', '관객', '돌출', '부착', '규칙', '에머', '자기', '죄목', '주인공', '이런', '아니', '베일', '아무', '1998', '이메일', '테이프', '위하', '매체', '착하', '설치', '카프카', '시키', '찍히', '스스로', '바뀌', '주변', '경악', '무렵', '대답', '라는', '니스', '이를테면',

In [94]:
total = 0
feature1 = featureVectorizer(morphs1)
feature2 = featureVectorizer(morphs2)

print(feature1)
print(feature2)

[0.1111111111111111, 0.0, 0.0, 0.1111111111111111, 0.0, 0.6666666666666666, 0.0, 0.1111111111111111]
[0.07142857142857142, 0.07142857142857142, 0.07142857142857142, 0.21428571428571427, 0.0, 0.35714285714285715, 0.0, 0.21428571428571427]


<b>Cosine distance 구하는 법:

In [95]:
result = 1 - spatial.distance.cosine(feature1, feature2)
result

0.8735555046022884